In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

In [2]:
path = '~/code/Strive/predict-future-sales/'

df_train = pd.read_csv(path + 'sales_train.csv')
# df_test = pd.read_csv(path + '')
df_categories = pd.read_csv(path + 'categories_en.csv')
# df_test = pd.read_csv(path + '')
df_shops = pd.read_csv(path + 'shops_en.csv')
df_items = pd.read_csv(path + 'items_en.csv')

df_shops

,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 francs",0
1,"! Yakutsk TC ""Central"" fran",1
2,"Adygea TC ""Mega""",2
3,"Balashikha TC ""Oktyabr-Kinomir""",3
4,"Volga TC ""Volga Mall""",4
5,"Vologda SEC ""Marmelad""",5
6,"Voronezh (Plekhanovskaya, 13)",6
7,"Voronezh SEC ""Maksimir""",7
8,"Voronezh SEC City-Park ""Grad""",8
9,Outbound Trade,9


In [3]:
#  manually dropping duplicates (0 - 57, 10 - 11, 39 - 40)



In [4]:
# Cleaning the shop_name column
df_shops['cleaned'] = [df_shops['shop_name'][i].replace('!', '').replace('?', '')
    .replace('²', '').replace('" ', ' ').replace(' "', ' ').replace('"', '')
    .replace('-', ' ').strip() for i in range(df_shops.shape[0])]

# Extracting the city
city = []
for i in range(df_shops.shape[0]):
    if df_shops['cleaned'][i].split()[0] == 'St.':
        city.append(df_shops['cleaned'][i].split()[0] + ' ' + df_shops['cleaned'][i].split()[1])
    # df_shops['cleaned'][39].split()[0][:6] == 'Rostov'
    elif df_shops['cleaned'][i].split()[0][:6] == 'Rostov':
        city.append(df_shops['cleaned'][i].split()[0][:6] + ' ' + df_shops['cleaned'][i].split()[0][6:8] + ' ' + df_shops['cleaned'][i].split()[0][8:])
    else:
        city.append(df_shops['cleaned'][i].split()[0])

df_shops['city'] = pd.DataFrame(city)

# correct a city name (duplicate)

df_shops['city'][40] = df_shops['city'][41] = df_shops['city'][39]

# classifing as on-line
is_online = []
for i in range(df_shops.shape[0]):
    if df_shops['city'][i] == 'Internet' or df_shops['city'][i] == 'Digital' or df_shops['city'][i] == 'Outbound':
        is_online.append(1)
    else:
        is_online.append(0)

df_shops['is_online'] = pd.DataFrame(is_online)

df_shops

<ipython-input-4-73483bb1943d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shops['city'][40] = df_shops['city'][41] = df_shops['city'][39]


,shop_name,shop_id,cleaned,city,is_online
0,"! Yakutsk Ordzhonikidze, 56 francs",0,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,0
1,"! Yakutsk TC ""Central"" fran",1,Yakutsk TC Central fran,Yakutsk,0
2,"Adygea TC ""Mega""",2,Adygea TC Mega,Adygea,0
3,"Balashikha TC ""Oktyabr-Kinomir""",3,Balashikha TC Oktyabr Kinomir,Balashikha,0
4,"Volga TC ""Volga Mall""",4,Volga TC Volga Mall,Volga,0
5,"Vologda SEC ""Marmelad""",5,Vologda SEC Marmelad,Vologda,0
6,"Voronezh (Plekhanovskaya, 13)",6,"Voronezh (Plekhanovskaya, 13)",Voronezh,0
7,"Voronezh SEC ""Maksimir""",7,Voronezh SEC Maksimir,Voronezh,0
8,"Voronezh SEC City-Park ""Grad""",8,Voronezh SEC City Park Grad,Voronezh,0
9,Outbound Trade,9,Outbound Trade,Outbound,1


In [5]:
# # city = df_shops['city'][0]
# location = geolocator.geocode('Rostov Na Donu')
# latitude = location.latitude
# longitude = location.longitude
# print(latitude, longitude)
# print(location)

In [6]:

# Longitude and Latitude

geolocator = Nominatim(user_agent="foursquare_agent")

coords = []
for i, city in enumerate(df_shops['city']):
    if df_shops['is_online'][i] == 0:
        # coords.append(city)
        try:
            location = geolocator.geocode(city)
            coords.append((location.latitude, location.longitude))
        except AttributeError:
            coords.append((i, 'error'))
    else:
        coords.append((np.nan, np.nan))

df_shops['latitude'] = [coords[i][0] if (coords[i] != np.nan) else np.nan for i in range(len(coords))]
df_shops['longitude'] = [coords[i][1] if (coords[i] != np.nan) else np.nan for i in range(len(coords))]

df_shops


,shop_name,shop_id,cleaned,city,is_online,latitude,longitude
0,"! Yakutsk Ordzhonikidze, 56 francs",0,"Yakutsk Ordzhonikidze, 56 francs",Yakutsk,0,62.027287,129.732086
1,"! Yakutsk TC ""Central"" fran",1,Yakutsk TC Central fran,Yakutsk,0,62.027287,129.732086
2,"Adygea TC ""Mega""",2,Adygea TC Mega,Adygea,0,44.693901,40.152042
3,"Balashikha TC ""Oktyabr-Kinomir""",3,Balashikha TC Oktyabr Kinomir,Balashikha,0,55.803623,37.964649
4,"Volga TC ""Volga Mall""",4,Volga TC Volga Mall,Volga,0,56.167024,44.215057
5,"Vologda SEC ""Marmelad""",5,Vologda SEC Marmelad,Vologda,0,59.218876,39.893276
6,"Voronezh (Plekhanovskaya, 13)",6,"Voronezh (Plekhanovskaya, 13)",Voronezh,0,51.660598,39.200586
7,"Voronezh SEC ""Maksimir""",7,Voronezh SEC Maksimir,Voronezh,0,51.660598,39.200586
8,"Voronezh SEC City-Park ""Grad""",8,Voronezh SEC City Park Grad,Voronezh,0,51.660598,39.200586
9,Outbound Trade,9,Outbound Trade,Outbound,1,NaN,NaN
